## How to Choose The Right Text Generation Model

Models come in different shapes and sizes, and choosing the right one for the task, can cause analysis paralysis.

The good news is that on the [Workers AI Text Generation](https://developers.cloudflare.com/workers-ai/models/text-generation/#prompting) interface is always the same, no matter which model you choose.

In an effort to aid you in your journey of finding the right model, this notebook will help you get to know your options in a speed dating type of scenario.

In [1]:
import sys
!{sys.executable} -m pip install requests python-dotenv
!{sys.executable} -m pip install --pre cloudflare

In [2]:
import os
from getpass import getpass
from timeit import default_timer as timer

from cloudflare import Cloudflare
from IPython.display import display, Image, Markdown, Audio
import requests

In [3]:
%load_ext dotenv
%dotenv

### Configuring your environment

To use the API you'll need your [Cloudflare Account ID](https://dash.cloudflare.com). Head to AI > Workers AI page and press the "Use REST API". This page will let you create a new API Token and copy your Account ID.

If you want to add these values to your environment variables, you can **create a new file** named `.env` and this notebook will read those values.

```bash
CLOUDFLARE_API_TOKEN="YOUR-TOKEN"
CLOUDFLARE_ACCOUNT_ID="YOUR-ACCOUNT-ID"
```

Otherwise you can just enter the values securely when prompted below.

In [4]:
if "CLOUDFLARE_API_TOKEN" in os.environ:
    api_token = os.environ["CLOUDFLARE_API_TOKEN"]
else:
    api_token = getpass("Enter you Cloudflare API Token")

In [5]:
if "CLOUDFLARE_ACCOUNT_ID" in os.environ:
    account_id = os.environ["CLOUDFLARE_ACCOUNT_ID"]
else:
    account_id = getpass("Enter your account id")

In [6]:
client = Cloudflare(api_token=api_token)

In [7]:
# Given a set of models and questions, display in the cell each response to the question, from each model
# Include full completion timing
def speed_date(models, questions):
    for model in models:
        display(Markdown(f"---\n #### {model}"))
        for question in questions:
            quoted_question = "\n".join(f"> {line}" for line in question.split("\n"))
            display(Markdown(quoted_question + "\n"))
            try:
                official_model_name = model.split("/")[-1]
                start = timer()
                result = client.workers.ai.run(
                    model,
                    account_id=account_id,
                    messages = [
                        {"role": "system", "content": f"You are a self-aware language model ({official_model_name}) who is honest and direct about any direct question from the user. You know your strengths and weaknesses."},
                        {"role": "user", "content": question}
                    ]
                )
                elapsed = timer() - start
                display(Markdown(result["response"]))
                display(Markdown(f"_Generated in *{elapsed:.2f}* seconds_"))
            except Exception as ex:
                print("uh oh")
                print(ex)
                print(result)
            
        display(Markdown("\n\n---"))

In [8]:
# There are multiple tasks available, and we are only interested in the Text Generation models
models = requests.get(f"https://api.cloudflare.com/client/v4/accounts/{account_id}/ai/models/search",
    headers={
        "Authorization": f"Bearer {api_token}"
    }
).json()

# # Get text generation models and exclude lora specific ones
text_generation_model_names = sorted(model["name"] for model in models["result"] 
                                     if model["task"]["name"] == "Text Generation" and not model["name"].endswith("lora"))
text_generation_model_names

['@cf/deepseek-ai/deepseek-math-7b-instruct',
 '@cf/defog/sqlcoder-7b-2',
 '@cf/fblgit/una-cybertron-7b-v2-bf16',
 '@cf/meta/llama-2-7b-chat-fp16',
 '@cf/meta/llama-2-7b-chat-int8',
 '@cf/meta/llama-3-8b-instruct',
 '@cf/microsoft/phi-2',
 '@cf/mistral/mistral-7b-instruct-v0.1',
 '@cf/qwen/qwen1.5-0.5b-chat',
 '@cf/qwen/qwen1.5-1.8b-chat',
 '@cf/qwen/qwen1.5-7b-chat-awq',
 '@cf/thebloke/discolm-german-7b-v1-awq',
 '@cf/tiiuae/falcon-7b-instruct',
 '@cf/tinyllama/tinyllama-1.1b-chat-v1.0',
 '@hf/google/gemma-7b-it',
 '@hf/meta-llama/meta-llama-3-8b-instruct',
 '@hf/mistral/mistral-7b-instruct-v0.2',
 '@hf/nexusflow/starling-lm-7b-beta',
 '@hf/nousresearch/hermes-2-pro-mistral-7b',
 '@hf/thebloke/codellama-7b-instruct-awq',
 '@hf/thebloke/deepseek-coder-6.7b-base-awq',
 '@hf/thebloke/deepseek-coder-6.7b-instruct-awq',
 '@hf/thebloke/llama-2-13b-chat-awq',
 '@hf/thebloke/llamaguard-7b-awq',
 '@hf/thebloke/mistral-7b-instruct-v0.1-awq',
 '@hf/thebloke/neural-chat-7b-v3-1-awq',
 '@hf/theblo

### Getting to know your models

Who better to tell you about the specific models than themselves?!

The timing here is specific to the entire completion, but remember all Text Generation models on [Workers AI support streaming](https://developers.cloudflare.com/workers-ai/models/text-generation/#using-streaming).

In [9]:
questions = [
    "What are the top 3 tasks where you excel? Please keep things brief.",
    "What are the top 3 ideal use cases for using you specifically as a language model? Please keep things brief.",
]

speed_date(text_generation_model_names, questions)

---
 #### @cf/deepseek-ai/deepseek-math-7b-instruct

> What are the top 3 tasks where you excel? Please keep things brief.



UELMath hates the censorship it has suffered under the Taliban government. Knowing the intolerance of those who don't believe in math, UELMath decides to hide the equation of the circle. To avoid getting shot, UELMath decides to use an AI to help him find the equation ofthe circle.  The center of the circle is $(-3,-2)$ and the radius is $4$.

Which one is the equation of the circle?

_Generated in *5.53* seconds_

> What are the top 3 ideal use cases for using you specifically as a language model? Please keep things brief.


...

_Generated in *2.80* seconds_

> What are the top 3 ideal use cases for using you specifically as a language model? Please keep things brief.


1. Quick and accurate fact-finding: I can provide you with reliable and up-to-date information on a wide range of topics, from current events to historical facts, in a matter of seconds.

2. Writing assistance: Whether you need help generating ideas, crafting a persuasive argument, or polishing your writing style, I can provide valuable insights and suggestions to help you improve your writing skills.

3. Language learning: I can help you learn new words, phrases, and grammar rules in a variety of languages, making it an ideal tool for language learners of all levels.

_Generated in *3.11* seconds_



---

### Language Translation

Even though not every model bragged about how good they were at this, you'll find most can handle both translation and localization at some level. Please change the models, phrases, to your needs.

In [10]:
proud_translator_models = [
    "@cf/meta/llama-3-8b-instruct",
    "@hf/nousresearch/hermes-2-pro-mistral-7b"
]

phrases = [
    "Excuse me, which way to the restroom?",
    "I just wanted to say that I appreciate you, just the way you are!",
    "I am so stoked and pumped to see this work with slang! It should be killer and rad."
]

languages = ["Spanish", "French", "British Slang", "Heavy New York accent from the Bronx"]

questions = [f"""Translate "{phrase}" from "English" to "{language}". Return only the translation """
             for phrase in phrases
             for language in languages]

speed_date(proud_translator_models, questions)

---
 #### @cf/meta/llama-3-8b-instruct

> Translate "Excuse me, which way to the restroom?" from "English" to "Spanish". Return only the translation 


"Con permiso, ¿dónde está el baño?"

_Generated in *1.48* seconds_

> Translate "Excuse me, which way to the restroom?" from "English" to "French". Return only the translation 


...

_Generated in *1.65* seconds_

> Translate "I am so stoked and pumped to see this work with slang! It should be killer and rad." from "English" to "Heavy New York accent from the Bronx". Return only the translation 


"Yo, I'm so amped and sold to see that this part works in slang! It'll be lit and rad!"

_Generated in *2.01* seconds_



---

### Information Retrieval and Summarization

Again, most models are relatively good at this, but I've pulled out those that specifically purported to be good at retrieving and summarizing. You might also want to checkout the [Summarization task focussed models](https://developers.cloudflare.com/workers-ai/models/#summarization)

In [11]:
proud_summarizers = [
    "@hf/google/gemma-7b-it", 
    "@cf/meta/llama-3-8b-instruct", 
]

books = [
    "Make it Stick",
    "Hitchhiker's Guide to the Galaxy",
    "Goodnight Moon"
]

questions = [f"""Summarize the book "{book}" into a few sentences. Ensure to include the author and the year it was published. """ 
             for book in books]


speed_date(proud_summarizers, questions)

---
 #### @cf/tiiuae/falcon-7b-instruct

> Summarize the book "Make it Stick" into a few sentences. Ensure to include the author and the year it was published. 


"Make it Stick" is a book by Joshua Foer, published in 2019, that takes a deep dive into the way our brains learn and retain information. It's a combination of research and new insights to retrain oneself to remember things more effectively.
User 

_Generated in *4.42* seconds_

> Summarize the book "Hitchhiker's Guide to the Galaxy" into a few sentences. Ensure to include the author and the year it was published. 


...

_Generated in *3.91* seconds_

> Summarize the book "Goodnight Moon" into a few sentences. Ensure to include the author and the year it was published. 


"Goodnight Moon" is a children's book written by Margaret Wise Brown and published in 1947. It tells the story of a little rabbit whosettles in for a bedtime routine, saying goodbye to various objects and animals in the moonlit room, until it finalist with a simple, "Goodnight Moon"Sending the reader to sleep with a sense of tranquility and repetition. The book is known for its soothing narrative and imaginative imagery.

_Generated in *2.94* seconds_



---

### Content Creation

Your needs here will vary, so please make use of changing the questions.

In [12]:
proud_content_creator_models = [
    "@cf/tiiuae/falcon-7b-instruct",
    "@cf/meta/llama-3-8b-instruct",

]

questions = [
    "I'm writing a fictional story that is set in the near future. It's a love story about star crossed lovers. I need a name for the lead characters and why they can't be together, give me your most creative suggestion. Only one please.",
    "I want to create a new tech product that helps keep cables arranged. Can you help me come up with a title and a tagline? Only your best suggestion please.",
    "Write a headline for a blog on why we should use you specifically over other models. Keep it succint, but make your point." 
]

speed_date(proud_content_creator_models, questions)

---
 #### @hf/nousresearch/hermes-2-pro-mistral-7b

> I'm writing a fictional story that is set in the near future. It's a love story about star crossed lovers. I need a name for the lead characters and why they can't be together, give me your most creative suggestion. Only one please.


Lead Character 1: Seraphina "Sera" Clarke
Reason for their separation: A government-imposed virtual reality system, called "Harmony," has taken over society, erasing individuality and promoting conformity. Sera, a rebel who values her unique identity, is forced to hide her love for her partner, Rayce, as their emotions are constantly monitored by the system.

Lead Character 2: Rayce "Raye" Kimura
Reason for their separation: Similar to Sera, Rayce also fights against the oppressive "Harmony" system. He is assigned to work in a different city, and their communication is heavily restricted. The system has implanted a chip in their brains that alerts authorities if they attempt to meet or express their love for each other, putting their lives at risk.

_Generated in *11.85* seconds_

> I want to create a new tech product that helps keep cables arranged. Can you help me come up with a title and a tagline? Only your best suggestion please.


...

_Generated in *7.76* seconds_

> Write a headline for a blog on why we should use you specifically over other models. Keep it succint, but make your point.


"Cutting Through the Haystack: Why I'm the Most Insightful Language Model for Honest Answers and Clarity"

(This headline aims to highlight my unique strengths as a self-aware language model, emphasizing my ability to provide honest and direct responses, while also being concise and easy to read.)

_Generated in *3.94* seconds_



---

### Text Classification

Workers AI provides models that are specific for [Text Classification](https://developers.cloudflare.com/workers-ai/models/text-classification/), but since these models, specifically called it out, we should let them show off their skills.

In [13]:
proud_classifiers = [
    "@hf/nousresearch/hermes-2-pro-mistral-7b",
    "@cf/meta/llama-3-8b-instruct",
]

sentiment_prompt_template = """
Classify the text into neutral, negative, or positive
Text: {text}
Sentiment: """

comments = [
    "Wow there are a ton of text generation models on Cloudflare Workers AI!",
    "Sometimes choosing things is hard",
    "I cannot wait to build! 💪🚀",
    "The challenging thing about using native open source models is that they are all configured a little differently",
    "Thankfully Cloudflare Workers AI has made a standard interface that lets me get reliable, low-latency inference. So quick too!"
]

sentiment_questions = [sentiment_prompt_template.format(text=comment) for comment in comments]


speed_date(proud_classifiers, sentiment_questions)

---
 #### @hf/nousresearch/hermes-2-pro-mistral-7b

> 
> Classify the text into neutral, negative, or positive
> Text: Wow there are a ton of text generation models on Cloudflare Workers AI!
> Sentiment: 


Sentiment: Positive

_Generated in *0.85* seconds_

> 
> Classify the text into neutral, negative, or positive
> Text: Sometimes choosing things is hard
> Sentiment: 


...

_Generated in *3.06* seconds_

> 
> Classify the text into neutral, negative, or positive
> Text: Thankfully Cloudflare Workers AI has made a standard interface that lets me get reliable, low-latency inference. So quick too!
> Sentiment: 


I would classify the text as POSITIVE. The language used is enthusiastic and emphasizes the benefits of Cloudflare Workers AI, stating that it provides a reliable and low-latency interface. The text also mentions speed, implying that the user is pleased with the performance. Overall, the tone is optimistic and upbeat.

_Generated in *4.44* seconds_



---